In [ ]:
# Get the stock quote from July 2015 to December 2020
# Pulled data from Yahoo Finance
import math
import pandas_datareader as web
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd

In [ ]:
import pandas_bokeh
pandas_bokeh.output_notebook()

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import Video
# Embed the video (replace 'your_video.mp4' with your actual video path)
Video("video/pl_readme_gif_2_0.mp4", embed=True)

In [ ]:
Video("video/pt_dm_vid.mp4", embed=True)

In [ ]:
forex_path = "FOREX/"
import os

In [ ]:
start_period = "2018-01-01"
end_period = "2024-01-01"
fig_size = (1500,400)

## Getting Yahoo Finance 
### GET THE DATE FOR MICROSOFT; APPLE; NVIDIA; INTEL

In [ ]:
import yfinance as yf
Image(url= "https://www.labsterx.com/wp-content/uploads/2017/03/yahoo-finance-summary.png")


In [ ]:
df_all = yf.download(("MSFT", "AAPL","NVDA", "INTC", "AMD"), start=start_period, end=end_period)


In [ ]:
df_all.to_feather("data/yahoo.fa")

In [ ]:
from IPython.display import display, HTML

display(HTML(df_all.head().to_html()))

In [ ]:
df_all = df_all.Close
display(HTML(df_all.head().to_html()))

In [ ]:
df_all.plot_bokeh(y = "NVDA", figsize = (1500,400))

In [ ]:
import seaborn as sb
# Plotting correlation heatmap
dataplot = sb.heatmap(df_all.corr(numeric_only=True), cmap="YlGnBu", annot=True)



In [ ]:
df_NVIDIA = pd.DataFrame(df_all["NVDA"])
display(HTML(df_NVIDIA.head().to_html()))
df_NVIDIA.plot_bokeh(figsize = fig_size)

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Assuming 'data' is the time series data
result = adfuller(df_NVIDIA)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

    ADF Statistic: The test statistic is compared to critical values at various significance levels (typically 1%, 5%, and 10%). If the ADF statistic is less than the critical value, then we reject the null hypothesis, implying the series is stationary.

    Null Hypothesis (H₀): The series has a unit root (i.e., it is non-stationary).

    Alternative Hypothesis (H₁): The series is stationary (i.e., no unit root).
This strongly suggests that the time series is non-stationary, meaning it likely has a trend, seasonality, or some other structure that violates the assumption of stationarity.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
fig = plot_acf(df_NVIDIA, lags=80)
fig.set_figwidth(20)

In [ ]:
from scipy.signal import detrend

detrended = detrend(df_NVIDIA.iloc[:, 0], type='constant')
detrended = pd.DataFrame(detrended, index=df_NVIDIA.index)
pd.concat([df_NVIDIA,detrended]).plot_bokeh(figsize = fig_size)

In [ ]:
detrended = detrend(df_NVIDIA.iloc[:, 0], type='linear')
detrended = pd.DataFrame(detrended, index=df_NVIDIA.index)
detrended.columns= ["DETREND"]
pd.concat([df_NVIDIA,detrended]).plot_bokeh(figsize = fig_size)

In [ ]:
rolling_mean = df_NVIDIA.rolling(window=60, center=True).mean()
detrended = pd.DataFrame(df_NVIDIA - rolling_mean)
detrended.columns = ['DETREND MA']

pd.concat([df_NVIDIA,detrended]).plot_bokeh(figsize = fig_size)

In [ ]:
df_prepared = detrended.dropna().iloc[:,0]
df_prepared
Image(url= "https://storage.googleapis.com/lightning-avatars/litpages/01hhda2ban5mpa8sa8gv0985cp/lstm_image.001.png")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pytorch_lightning as pl

In [ ]:
# Custom Dataset for LSTM with multi-step ahead prediction
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length, steps_ahead):
        self.data = data
        self.seq_length = seq_length
        self.steps_ahead = steps_ahead

    def __len__(self):
        return len(self.data) - self.seq_length - self.steps_ahead + 1

    def __getitem__(self, index):
        # Get the sequence of input data
        x = self.data[index:index + self.seq_length]
        # Get the target as the next 'steps_ahead' values
        y = self.data[index + self.seq_length : index + self.seq_length + self.steps_ahead]
        return torch.FloatTensor(x), torch.FloatTensor(y)

In [ ]:
# PyTorch Lightning DataModule
class StockDataModule(pl.LightningDataModule):
    def __init__(self, df, seq_length=5, steps_ahead=1, batch_size=32, split_ratio=0.8):
        super().__init__()
        self.df = df
        self.seq_length = seq_length
        self.steps_ahead = steps_ahead
        self.batch_size = batch_size
        self.split_ratio = split_ratio
        self.scaler = MinMaxScaler()

    def setup(self, stage=None):
        # Extract the stock prices and scale the data
        data = self.df.values.reshape(-1, 1)
        data = self.scaler.fit_transform(data)
        data = data.flatten()

        # Split data into training and test
        train_size = int(len(data) * self.split_ratio)
        self.train_data = data[:train_size]
        self.val_data = data[train_size:]

    def train_dataloader(self):
        train_dataset = TimeSeriesDataset(self.train_data, self.seq_length, self.steps_ahead)
        return DataLoader(train_dataset, batch_size=self.batch_size, num_workers=11, shuffle=True)

    def val_dataloader(self):
        val_dataset = TimeSeriesDataset(self.val_data, self.seq_length, self.steps_ahead)
        return DataLoader(val_dataset, batch_size=self.batch_size, num_workers=11 )

    def test_dataloader(self):
        test_dataset = TimeSeriesDataset(self.val_data, self.seq_length, self.steps_ahead)  # Using val data for test
        return DataLoader(test_dataset, batch_size=self.batch_size, num_workers=11)
    
    def predict_dataloader(self):
        return self.test_dataloader()

    

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
# Define the LSTM model for multi-step ahead prediction
class LSTMForecastingModel(pl.LightningModule):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, steps_ahead=1, lr=1e-3):
        super(LSTMForecastingModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.steps_ahead = steps_ahead
        self.lr = lr

        # Define LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define a fully connected layer to output the steps_ahead predictions
        self.fc = nn.Linear(hidden_size, steps_ahead)

    def forward(self, x):
        if len(x.shape) == 2:  # Input shape could be [seq_length, input_size] without batch
            x = x.unsqueeze(1)
        # Initialize hidden and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pass the input through the LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        # Take the last output from the LSTM and pass it through the fully connected layer
        out = self.fc(out[:,-1])
        
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('test_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    # Function to predict for test dataset
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, _ = batch
        return self(x)

In [ ]:
# Instantiate the DataModule with your desired number of steps ahead
seq_length = 30
steps_ahead = 5
data_module = StockDataModule(df_prepared, seq_length=seq_length, steps_ahead=steps_ahead, batch_size=32)

# Instantiate the LSTM model for 3-step ahead forecasting
model = LSTMForecastingModel(input_size=seq_length, hidden_size=64, num_layers=2, steps_ahead=steps_ahead, lr=1e-3)

# Define a PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=10)

# Train the model
trainer.fit(model, data_module)

In [ ]:
# Validate the model
trainer.validate(model, data_module)

# Test the model
trainer.test(model, data_module)

# Predict on test data
predictions = trainer.predict(model, data_module)

In [ ]:
import pandas as pd
import torch
def predict_model(model, trainer,data_module):
    # Get the number of steps ahead the model is predicting
    steps_ahead = model.steps_ahead

    # Get the predictions for the test set (a list of tensors, each with shape [batch_size, steps_ahead])
    predictions = trainer.predict(model, data_module)

    # Concatenate the list of predictions into a single tensor (shape: [num_samples, steps_ahead])
    predictions = torch.cat(predictions).cpu().detach().numpy()

    # Get the actual target values from the test dataloader
    actuals = []
    test_dataloader = data_module.test_dataloader()
    for batch in test_dataloader:
        _, y = batch
        actuals.append(y)

    # Concatenate the actual values into a single tensor (shape: [num_samples, steps_ahead])
    actuals = torch.cat(actuals).cpu().detach().numpy()

    # Prepare an empty DataFrame to hold results with enough space for shifting
    df_results = pd.DataFrame(index=range(len(predictions)))

    # Populate the DataFrame with shifted predictions and actuals
    for step in range(steps_ahead):
        # Shift the predictions upwards, since they refer to future steps
        df_results[f'Prediction_Step_{step + 1}'] = pd.Series(predictions[:, step]).shift(-step)
        # Actuals do not need to be shifted
        df_results[f'Actual_Step_{step + 1}'] = pd.Series(actuals[:, step])

    # Create MultiIndex for columns (Step first, then Type)
    columns = pd.MultiIndex.from_product(
        [[f'Step_{i+1}' for i in range(steps_ahead)], ['Prediction', 'Actual']],
        names=['Step', 'Type']
    )

    # Rearrange the DataFrame with MultiIndex columns
    df_results.columns = columns

    # Drop rows with NaN values caused by shifting
    df_results.dropna(inplace=True)
    return df_results



In [ ]:
predict_model(model, trainer,data_module)["Step_5"].plot_bokeh(figsize = fig_size)

In [ ]:

# Instantiate the DataModule with your desired number of steps ahead
seq_length = 50
steps_ahead = 10
data_module = StockDataModule(df_NVIDIA, seq_length=seq_length, steps_ahead=steps_ahead, batch_size=32)

# Instantiate the LSTM model for 3-step ahead forecasting
model = LSTMForecastingModel(input_size=seq_length, hidden_size=64, num_layers=2, steps_ahead=steps_ahead, lr=1e-3)

# Define a PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=100)

# Train the model
trainer.fit(model, data_module)

# Validate the model
trainer.validate(model, data_module)

# Test the model
trainer.test(model, data_module)

# Predict on test data
predictions = trainer.predict(model, data_module)


In [ ]:
predict_model(model, trainer,data_module)["Step_1"].plot_bokeh(figsize = fig_size)


In [ ]:
import os
forex_path = "data/FOREX"
df = pd.read_csv(os.path.join(forex_path, "2019.csv"), header=None)
df.head()

In [ ]:
# Create an empty list to store dataframes (each dataframe will be from a single CSV file)
dataframes = []

# Loop through all the files in the folder
for filename in os.listdir(forex_path):
    
    # Check if the file ends with ".csv" (so we only process CSV files)
    if filename.endswith(".csv"):
        file_path = os.path.join(forex_path, filename)  # Create the full path to the file

        # Read the CSV file into a pandas dataframe
        # 'parse_dates=[0]' tells pandas to interpret the first column as dates
        # 'index_col=0' makes the first column the index of the dataframe (Datetime index)
        df = pd.read_csv(file_path, parse_dates=[0], index_col=0, sep=";", header = None)
        # We assume the second column has the values we need (this could vary based on your files)
        # We only keep the second column (index 0 for the first, 1 for the second column)
        df = df.iloc[:, [0]]  # This keeps only the second column (values)
        # Drop duplicate indices (duplicate dates)
        df = df[~df.index.duplicated(keep='first')]


        # Append this dataframe to our list of dataframes
        dataframes.append(df)

# After we've gone through all the CSV files and created dataframes,
# we concatenate (join) them all together into one dataframe along the 'Datetime' index
# This means the dataframes will be aligned by their dates (the Datetime index)
if dataframes:
    joined_df = pd.concat(dataframes, axis=0)
else:
    print("No CSV files found in the folder.")

# Display the joined dataframe

joined_df= joined_df.sort_index()


In [ ]:
joined_df.columns = ["USD"]

In [ ]:
# Assuming your current dataframe is named `full_df` and has a DatetimeIndex with minute-level data
# Ensure 'full_df' has 'value' column with exchange rate data

# Step 1: Define the complete date range from 2019-01-01 00:00 to 2023-12-31 23:59
full_date_range = pd.date_range(start='2019-01-01 00:00', end='2023-12-31 23:59', freq='min')

# Step 2: Reindex your dataframe to this full date range
# This will introduce NaN values for any missing minutes
full_df_reindexed = joined_df.reindex(full_date_range)


# Step 3: Fill missing data at the beginning and end with forward-fill and backward-fill
# First, use forward-fill for missing data at the beginning, then backward-fill for the end
full_df_reindexed['USD'] = full_df_reindexed['USD'].ffill().bfill()

# Step 3: Use spline interpolation (order=3 for cubic interpolation)
# You can also experiment with other orders like 2 (quadratic) or higher
full_df_reindexed['USD'] = full_df_reindexed['USD'].interpolate(method='spline', order=3)


In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.preprocessing import MinMaxScaler


class MinuteToDayDataset(Dataset):
    def __init__(self, df, scaler=None):
        """
        Dataset for loading minute-level time series data grouped into daily chunks (1440 minutes per day).
        
        Args:
        df (pandas.DataFrame): DataFrame with minute-level data and a 'value' column.
        scaler (sklearn.preprocessing): Optional scaler for scaling the data.
        """
        self.df = df
        self.scaler = scaler

        # Ensure that the dataframe index is a datetime index
        if not pd.api.types.is_datetime64_any_dtype(df.index):
            raise ValueError("The dataframe index must be of datetime type.")
        self.df.columns = ["USD"]

        # Apply scaling if a scaler is provided
        self.daily_data = list(self.df.groupby(self.df.index.floor("d")))
        # Apply scaling if a scaler is provided
        if self.scaler is not None:
            self.scale_data()
            
    def scale_data(self):
        """
        Scales the daily data using the provided scaler.
        """
        for i in range(len(self.daily_data)):
            self.daily_data[i][1]['USD'] = self.scaler.transform(self.daily_data[i][1]['USD'].values.reshape(-1, 1))

    
    def __len__(self):
        # The number of samples is equal to the number of full days in the data
        return len(self.daily_data)
    
    def __getitem__(self, idx):
        # Get the daily data at index `idx` and return the 'value' column as a numpy array
        day_df = self.daily_data[idx][1]
        return day_df['USD'].values.astype(np.float32),  idx


In [ ]:
# Step 2: PyTorch Lightning DataModule to manage the dataset and DataLoaders
class ExchangeRateDataModule(pl.LightningDataModule):
    def __init__(self, df, batch_size=32, train_val_test_split=(0.7, 0.15, 0.15)):
        """
        DataModule to handle the loading and splitting of the dataset for train, validation, and test phases.
        
        Args:
        df (pandas.DataFrame): DataFrame containing the exchange rate data.
        batch_size (int): Batch size for DataLoaders.
        train_val_test_split (tuple): Ratio for splitting the dataset into train, validation, and test sets.
        """
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.train_val_test_split = train_val_test_split
        self.scaler = MinMaxScaler()  # MinMaxScaler to scale between 0 and 1

    def setup(self, stage=None):
        """
        Sets up the train, validation, and test datasets by splitting the original dataset.
        This function is automatically called by PyTorch Lightning when appropriate.
        """
        # Fit the scaler on the full dataset
        self.scaler.fit(self.df['USD'].values.reshape(-1, 1))

        # Create the dataset with daily groups and scaled data
        full_dataset = MinuteToDayDataset(self.df, scaler=self.scaler)

        # Calculate train, validation, and test split sizes
        train_size = int(self.train_val_test_split[0] * len(full_dataset))
        val_size = int(self.train_val_test_split[1] * len(full_dataset))
        test_size = len(full_dataset) - train_size - val_size

        # Perform the split
        self.train_dataset, self.val_dataset, self.test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        # Return the DataLoader for the training set
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        # Return the DataLoader for the validation set
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        # Return the DataLoader for the test set
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import pandas as pd

class ConvAutoEncoder(pl.LightningModule):
    def __init__(self, input_length=1440, latent_dim=24, learning_rate = 0.001):
        super(ConvAutoEncoder, self).__init__()
        # Save hyperparameters to log them
        self.save_hyperparameters()
        
        self.latent_dim = latent_dim
        self.input_length = input_length
        self.learning_rate = learning_rate
        
        # Encoder: The layers that compress the data
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        
        # Latent space: Compress the sequence to a small, latent representation
        self.fc1 = nn.Linear(64 * (input_length // 8), self.latent_dim)
        self.fc2 = nn.Linear(self.latent_dim, 64 * (input_length // 8))
        
        # Decoder: The layers that reconstruct the data back to the original size
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=1, kernel_size=7, stride=2, padding=3, output_padding=1),
            nn.Sigmoid()  # Sigmoid to scale values between 0 and 1
        )

    def forward(self, x):
        # Forward pass through the encoder, latent space, and decoder
        x = x.unsqueeze(1)  # Add a channel dimension for Conv1d
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten for the fully connected layers
        
        # Latent space
        latent = self.fc1(x)
        x = self.fc2(latent)
        x = x.view(x.size(0), 64, -1)  # Reshape for the decoder
        
        # Decode the data
        reconstruction = self.decoder(x)
        return reconstruction.squeeze(1)  # Remove channel dimension

    def training_step(self, batch, batch_idx):
        # Training step: Calculate reconstruction loss (MSE) for training
        x, _ = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        # Validation step: Calculate reconstruction loss (MSE) for validation
        x, _ = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        # Test step: Calculate reconstruction loss (MSE) and store real and predicted values
        x, original_index = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('test_loss', loss)
        
        # Store original and reconstructed values for later
        self.real_values.append(x.cpu().detach().numpy())
        self.reconstructed_values.append(reconstruction.cpu().detach().numpy())
        self.real_indices.append(original_index)
        
        return loss

    def configure_optimizers(self):
        # Optimizer for the model
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
    
    def on_test_epoch_start(self):
        # Initialize lists to store original, reconstructed values, and their timestamps during testing
        self.real_values = []
        self.reconstructed_values = []
        self.real_indices = []

    def predict_step(self, batch, batch_idx):
        # Predict step: Runs inference on the input batch to generate reconstructed values
        x, original_index = batch
        reconstruction = self.forward(x)
        return reconstruction, x, original_index

    def predict_dataloader(self, dataloader):
        """
        This method performs predictions on the entire dataloader and returns the predictions and the real data in a DataFrame.
        """
        # Store all the real and predicted values
        all_reconstructed = []
        all_real = []
        all_indices = []

        # Run predictions
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            for batch in dataloader:
                reconstructed, real, index = self.predict_step(batch, None)
                all_reconstructed.append(reconstructed.cpu().numpy())
                all_real.append(real.cpu().numpy())
                all_indices.append(index)

        # Concatenate all batches
        all_reconstructed = np.concatenate(all_reconstructed, axis=0)
        all_real = np.concatenate(all_real, axis=0)
        all_indices = np.concatenate(all_indices, axis=0)

        # Create a DataFrame with original, reconstructed values, and their corresponding indices
        results_df = pd.DataFrame({
            'Date': all_indices.flatten(),
            'Real Value': all_real.flatten(),
            'Reconstructed Value': all_reconstructed.flatten()
        })

        return results_df
    # New `predict` method with inverse scaling
    def predict(self, datamodule, split='test'):
        """
        This method runs inference using the DataModule's dataloaders and reconstructs the full day.
        
        Args:
        datamodule: The PyTorch Lightning DataModule that contains dataloaders.
        split: Which dataloader to use (train, val, or test).
        
        Returns:
        A pandas DataFrame with the real values, reconstructed values, and full daily minute-level timestamps.
        """
        # Select the appropriate dataloader based on the split ('train', 'val', or 'test')
        if split == 'test':
            dataloader = datamodule.test_dataloader()
        elif split == 'val':
            dataloader = datamodule.val_dataloader()
        else:
            dataloader = datamodule.train_dataloader()

        # Store all the real and predicted values
        all_reconstructed = []
        all_real = []
        all_full_day_index = []

        # Run predictions
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient computation
            for batch in dataloader:
                x, original_index = batch
                reconstruction = self.forward(x)  # Run the forward pass to get predictions

                # Store original and reconstructed values
                all_reconstructed.append(reconstruction.cpu().numpy())
                all_real.append(x.cpu().numpy())


        # Concatenate all batches of reconstructed and real values
        all_reconstructed = np.concatenate(all_reconstructed, axis=0)
        all_real = np.concatenate(all_real, axis=0)

        # Inverse transform the reconstructed and real values to get them back to their original scale
        scaler = datamodule.scaler
        all_reconstructed = scaler.inverse_transform(all_reconstructed.reshape(-1, 1))
        all_real = scaler.inverse_transform(all_real.reshape(-1, 1))

        # Create a DataFrame with the original, reconstructed values, and their full day index
        results_df = pd.DataFrame({
            'Real Value': all_real.flatten(),
            'Reconstructed Value': all_reconstructed.flatten()
        })
        return results_df


In [ ]:

# Step 4: Load your data and initialize the DataModule
# Assuming 'full_df' is your dataframe with minute-level exchange rate data
data_module = ExchangeRateDataModule(df=full_df_reindexed, batch_size=32)

# Step 5: Initialize the AutoEncoder model
autoencoder = ConvAutoEncoder(input_length=1440, latent_dim = 24)

# Step 6: Train the model using PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=10)

# Step 7: Train and validate the model
trainer.fit(autoencoder, data_module)

# Step 8: Test the model
trainer.test(autoencoder, datamodule=data_module)

In [ ]:
start_index = i * autoencoder.input_le
results_df[0:1440]

In [ ]:
# Run prediction and get the results in a DataFrame
results_df = autoencoder.predict(data_module)
results_df.plot_bokeh(figsize = fig_size)

## MLFlow

In [ ]:
#pip install mlflow
from pytorch_lightning.loggers import MLFlowLogger
import mlflow

In [ ]:
mlflow_logger = MLFlowLogger(
    experiment_name='AutoencoderExperiment',
    run_name='ConvAutoEncoderRun',
    tracking_uri='file:./mlruns'  # Or use your actual MLflow tracking URI
)

In [ ]:
# Enable MLflow autologging for more detailed tracking (optional)
mlflow.pytorch.autolog()

In [ ]:
# Step 2: Load your data and initialize the DataModule
data_module = ExchangeRateDataModule(df=full_df_reindexed, batch_size=32)

# Step 3: Initialize the AutoEncoder model
autoencoder = ConvAutoEncoder(input_length=1440, latent_dim=24)

# Step 4: Train the model using PyTorch Lightning Trainer with MLflow logging
trainer = pl.Trainer(
    max_epochs=20, logger=mlflow_logger
)

In [ ]:
# Step 5: Train and validate the model
trainer.fit(autoencoder, data_module)

# Step 6: Test the model
trainer.test(autoencoder, datamodule=data_module)


In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import pandas as pd
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping, RichModelSummary, RichProgressBar
from torch.optim.lr_scheduler import ReduceLROnPlateau
class ConvAutoEncoder(pl.LightningModule):
    def __init__(self, input_length=1440, latent_dim=24, learning_rate = 0.001):
        super(ConvAutoEncoder, self).__init__()
        # Save hyperparameters to log them
        self.save_hyperparameters()
        
        self.latent_dim = latent_dim
        self.input_length = input_length
        self.learning_rate = learning_rate
        
        # Encoder: The layers that compress the data
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        
        # Latent space: Compress the sequence to a small, latent representation
        self.fc1 = nn.Linear(64 * (input_length // 8), self.latent_dim)
        self.fc2 = nn.Linear(self.latent_dim, 64 * (input_length // 8))
        
        # Decoder: The layers that reconstruct the data back to the original size
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=32, out_channels=16, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=16, out_channels=1, kernel_size=7, stride=2, padding=3, output_padding=1),
            nn.Sigmoid()  # Sigmoid to scale values between 0 and 1
        )

    def forward(self, x):
        # Forward pass through the encoder, latent space, and decoder
        x = x.unsqueeze(1)  # Add a channel dimension for Conv1d
        x = self.encoder(x)
        x = x.view(x.size(0), -1)  # Flatten for the fully connected layers
        
        # Latent space
        latent = self.fc1(x)
        x = self.fc2(latent)
        x = x.view(x.size(0), 64, -1)  # Reshape for the decoder
        
        # Decode the data
        reconstruction = self.decoder(x)
        return reconstruction.squeeze(1)  # Remove channel dimension

    def training_step(self, batch, batch_idx):
        # Training step: Calculate reconstruction loss (MSE) for training
        x, _ = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Validation step: Calculate reconstruction loss (MSE) for validation
        x, _ = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Test step: Calculate reconstruction loss (MSE) and store real and predicted values
        x, original_index = batch
        reconstruction = self.forward(x)
        loss = nn.MSELoss()(reconstruction, x)
        self.log('test_loss', loss)
        
        # Store original and reconstructed values for later
        self.real_values.append(x.cpu().detach().numpy())
        self.reconstructed_values.append(reconstruction.cpu().detach().numpy())
        self.real_indices.append(original_index)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        
        # Scheduler to reduce learning rate on plateau
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, min_lr=1e-6)
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',  # Reduce LR when val_loss plateaus
                'interval': 'epoch',
                'frequency': 1
            }
        }
    
    def on_test_epoch_start(self):
        # Initialize lists to store original, reconstructed values, and their timestamps during testing
        self.real_values = []
        self.reconstructed_values = []
        self.real_indices = []

    def predict_step(self, batch, batch_idx):
        # Predict step: Runs inference on the input batch to generate reconstructed values
        x, original_index = batch
        reconstruction = self.forward(x)
        return reconstruction, x, original_index

    def predict_dataloader(self, dataloader):
        """
        This method performs predictions on the entire dataloader and returns the predictions and the real data in a DataFrame.
        """
        # Store all the real and predicted values
        all_reconstructed = []
        all_real = []
        all_indices = []

        # Run predictions
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            for batch in dataloader:
                reconstructed, real, index = self.predict_step(batch, None)
                all_reconstructed.append(reconstructed.cpu().numpy())
                all_real.append(real.cpu().numpy())
                all_indices.append(index)

        # Concatenate all batches
        all_reconstructed = np.concatenate(all_reconstructed, axis=0)
        all_real = np.concatenate(all_real, axis=0)
        all_indices = np.concatenate(all_indices, axis=0)

        # Create a DataFrame with original, reconstructed values, and their corresponding indices
        results_df = pd.DataFrame({
            'Date': all_indices.flatten(),
            'Real Value': all_real.flatten(),
            'Reconstructed Value': all_reconstructed.flatten()
        })

        return results_df
    # New `predict` method with inverse scaling
    def predict(self, datamodule, split='test'):
        """
        This method runs inference using the DataModule's dataloaders and reconstructs the full day.
        
        Args:
        datamodule: The PyTorch Lightning DataModule that contains dataloaders.
        split: Which dataloader to use (train, val, or test).
        
        Returns:
        A pandas DataFrame with the real values, reconstructed values, and full daily minute-level timestamps.
        """
        # Select the appropriate dataloader based on the split ('train', 'val', or 'test')
        if split == 'test':
            dataloader = datamodule.test_dataloader()
        elif split == 'val':
            dataloader = datamodule.val_dataloader()
        else:
            dataloader = datamodule.train_dataloader()

        # Store all the real and predicted values
        all_reconstructed = []
        all_real = []
        all_full_day_index = []

        # Run predictions
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient computation
            for batch in dataloader:
                x, original_index = batch
                reconstruction = self.forward(x)  # Run the forward pass to get predictions

                # Store original and reconstructed values
                all_reconstructed.append(reconstruction.cpu().numpy())
                all_real.append(x.cpu().numpy())


        # Concatenate all batches of reconstructed and real values
        all_reconstructed = np.concatenate(all_reconstructed, axis=0)
        all_real = np.concatenate(all_real, axis=0)

        # Inverse transform the reconstructed and real values to get them back to their original scale
        scaler = datamodule.scaler
        all_reconstructed = scaler.inverse_transform(all_reconstructed.reshape(-1, 1))
        all_real = scaler.inverse_transform(all_real.reshape(-1, 1))

        # Create a DataFrame with the original, reconstructed values, and their full day index
        results_df = pd.DataFrame({
            'Real Value': all_real.flatten(),
            'Reconstructed Value': all_reconstructed.flatten()
        })
        return results_df


In [ ]:

# Step 1: Initialize MLflow logger
mlflow_logger = MLFlowLogger(
    experiment_name='AutoencoderExperiment',
    run_name='ConvAutoEncoderRun',
    tracking_uri='file:./mlruns'  # Adjust this based on your MLflow setup
)

# Step 2: Initialize the AutoEncoder model
autoencoder = ConvAutoEncoder(input_length=1440, latent_dim=24, learning_rate=0.001)

# Step 3: Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Metric to monitor
    dirpath='./checkpoints',  # Directory to save the model
    filename='best-model-{epoch:02d}-{val_loss:.2f}',  # Naming pattern
    save_top_k=1,  # Save only the best model
    mode='min',  # Minimize the validation loss
)

# Step 4: Define the LearningRateMonitor callback
lr_monitor = LearningRateMonitor(logging_interval='epoch')

# Step 5: Load your data and initialize the DataModule
data_module = ExchangeRateDataModule(df=full_df_reindexed, batch_size=32)
# Step 6: Train the model using PyTorch Lightning Trainer with Eartly stopping
early_stop_callback = EarlyStopping(
    monitor='val_loss',  # Monitorea el val_loss
    patience=5,  # Se detiene si no mejora después de 5 epochs
    mode='min'  # Detiene si no mejora
) 
rich_progress = RichProgressBar()
rich_summary = RichModelSummary(max_depth=2)  # Max depth is adjustable for how deep the summary should go

# Step 6: Train the model using PyTorch Lightning Trainer with MLflow logging
trainer = pl.Trainer(
    max_epochs=50,  # Increase epochs for better training
    logger=mlflow_logger,  # Pass the MLflow logger to the Trainer
    callbacks=[checkpoint_callback, lr_monitor, early_stop_callback, rich_progress, rich_summary],  # Add the callbacks for checkpoint and LR monitoring
    log_every_n_steps=10  # Optional: log metrics every 10 steps
)



# Step 7: Train and validate the model
trainer.fit(autoencoder, data_module)

# Step 8: Test the model
trainer.test(autoencoder, datamodule=data_module)

# Step 9: Enable MLflow autologging for more detailed tracking (optional)
mlflow.pytorch.autolog()


In [ ]:
# Function to train the model using Ray Tune
import ray
from ray import tune
from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback
def train_autoencoder(config, data_module):
    # Step 1: Initialize MLflow logger
    mlflow_logger = MLFlowLogger(
        experiment_name='AutoencoderExperiment',
        run_name='ConvAutoEncoderRun',
        tracking_uri='file:./mlruns'  # Adjust this based on your MLflow setup
    )

    # Step 2: Initialize the AutoEncoder model
    autoencoder = ConvAutoEncoder(input_length=config["input_length"], latent_dim=config["latent_dim"], learning_rate=config["learning_rate"])

    # Step 3: Define the ModelCheckpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',  # Metric to monitor
        dirpath='./checkpoints',  # Directory to save the model
        filename='best-model-{epoch:02d}-{val_loss:.2f}',  # Naming pattern
        save_top_k=1,  # Save only the best model
        mode='min',  # Minimize the validation loss
    )

    # Step 4: Define the LearningRateMonitor callback
    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    # Step 5: Load your data and initialize the DataModule
    data_module = ExchangeRateDataModule(df=full_df_reindexed, batch_size=32)
    # Step 6: Train the model using PyTorch Lightning Trainer with Eartly stopping
    early_stop_callback = EarlyStopping(
        monitor='val_loss',  # Monitorea el val_loss
        patience=5,  # Se detiene si no mejora después de 5 epochs
        mode='min'  # Detiene si no mejora
    ) 
    rich_progress = RichProgressBar()
    rich_summary = RichModelSummary(max_depth=2)  # Max depth is adjustable for how deep the summary should go

    # Step 6: Train the model using PyTorch Lightning Trainer with MLflow logging
    trainer = pl.Trainer(
        max_epochs=50,  # Increase epochs for better training
        logger=mlflow_logger,  # Pass the MLflow logger to the Trainer
        callbacks=[checkpoint_callback, lr_monitor, early_stop_callback, rich_progress, rich_summary],  # Add the callbacks for checkpoint and LR monitoring
        log_every_n_steps=10  # Optional: log metrics every 10 steps
    )



    # Step 7: Train and validate the model
    trainer.fit(autoencoder, data_module)

In [ ]:

# Define the hyperparameter search space
search_space = {
    "learning_rate": tune.loguniform(1e-5, 1e-3),
    "latent_dim": tune.choice([16, 24, 32]),
    "input_length": 1440  # Fixed in this case, but can be a tunable parameter if needed
}

# Initialize Ray Tune search
analysis = tune.run(
    tune.with_parameters(train_autoencoder, data_module=data_module),  # Pass the function and data module
    config=search_space,
    num_samples=10,  # Number of samples to run
    resources_per_trial={"cpu": 2, "gpu": 0},  # Adjust based on your system's resources
    metric="val_loss",  # Metric to optimize
    mode="min"  # We want to minimize validation loss
)

# Print the best hyperparameters found
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# Model that can be LSTM, GRU, or RNN depending on the network_type hyperparameter
class RNNForecastingModel(pl.LightningModule):
    def __init__(self, network_type='LSTM', input_size=1, hidden_size=64, num_layers=2, steps_ahead=1, lr=1e-3):
        super(RNNForecastingModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.steps_ahead = steps_ahead
        self.lr = lr
        self.network_type = network_type

        # Initialize RNN, GRU, or LSTM based on network_type
        if self.network_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif self.network_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        elif self.network_type == 'RNN':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid network_type. Choose from 'LSTM', 'GRU', or 'RNN'.")

        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, steps_ahead)

    def forward(self, x):
        if len(x.shape) == 2:  # If input shape is [seq_length, input_size] without batch dimension
            x = x.unsqueeze(1)
        
        # Initialize hidden state (and cell state if LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        if self.network_type == 'LSTM':
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            out, _ = self.rnn(x, (h0, c0))  # LSTM requires both hidden and cell state
        else:
            out, _ = self.rnn(x, h0)  # GRU and RNN only require hidden state

        # Pass the last output of the RNN/GRU/LSTM through the fully connected layer
        out = self.fc(out[:, -1, :])
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.MSELoss()(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# Function to run training with Ray Tune
def train_tune(config, data_module=None):
    model = RNNForecastingModel(
        network_type=config["network_type"],  # LSTM, GRU, or RNN
        input_size=config["input_size"],
        hidden_size=config["hidden_size"],
        num_layers=config["num_layers"],
        steps_ahead=config["steps_ahead"],
        lr=config["lr"]
    )

    trainer = pl.Trainer(
        max_epochs=10,
        gpus=1 if torch.cuda.is_available() else 0,
        logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version=""),
        callbacks=[
            TuneReportCallback({"val_loss": "val_loss"}, on="validation_end")
        ]
    )
    
    trainer.fit(model, data_module)

# Define the hyperparameter search space
config = {
    "network_type": tune.choice(["LSTM", "GRU", "RNN"]),  # Choose model type
    "input_size": tune.choice([30, 50]),  # Example input sequence lengths
    "hidden_size": tune.choice([64, 128]),  # Hidden layer size
    "num_layers": tune.choice([1, 2, 3]),  # Number of RNN layers
    "steps_ahead": 1,  # One step ahead forecasting (fixed in this case)
    "lr": tune.loguniform(1e-4, 1e-2)  # Learning rate search space
}

# Setup Ray Tune's scheduler and reporter
scheduler = ASHAScheduler(
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    parameter_columns=["network_type", "input_size", "hidden_size", "num_layers", "lr"],
    metric_columns=["val_loss", "training_iteration"]
)

# Launch the Ray Tune hyperparameter search
tune.run(
    tune.with_parameters(train_tune, data_module=data_module),
    resources_per_trial={"cpu": 2, "gpu": 1 if torch.cuda.is_available() else 0},
    metric="val_loss",
    mode="min",
    config=config,
    num_samples=10,  # Number of hyperparameter configurations to try
    scheduler=scheduler,
    progress_reporter=reporter
)


https://www.kaggle.com/datasets/utathya/future-volume-prediction

In [ ]:
from pytorch_forecasting.data.examples import get_stallion_data

df_stallion = get_stallion_data()

In [ ]:
df_stallion.head()

In [ ]:

# add time index
df_stallion["time_idx"] = df_stallion["date"].dt.year * 12 + df_stallion["date"].dt.month
df_stallion["time_idx"] -= df_stallion["time_idx"].min()

# add additional features
df_stallion["month"] = df_stallion.date.dt.month.astype(str).astype("category")  # categories have be strings
df_stallion["log_volume"] = np.log(df_stallion.volume + 1e-8)
df_stallion["avg_volume_by_sku"] = df_stallion.groupby(["time_idx", "sku"], observed=True).volume.transform("mean")
df_stallion["avg_volume_by_agency"] = df_stallion.groupby(["time_idx", "agency"], observed=True).volume.transform("mean")

# we want to encode special days as one variable and thus need to first reverse one-hot encoding
special_days = [
    "easter_day",
    "good_friday",
    "new_year",
    "christmas",
    "labor_day",
    "independence_day",
    "revolution_day_memorial",
    "regional_games",
    "fifa_u_17_world_cup",
    "football_gold_cup",
    "beer_capital",
    "music_fest",
]
df_stallion[special_days] = df_stallion[special_days].apply(lambda x: x.map({0: "-", 1: x.name})).astype("category")

In [ ]:
df_stallion.columns

In [ ]:
df_stallion["agency_number"] = df_stallion["agency"].str.extract('(\d+)')

# If you want to convert the extracted numbers to integer
df_stallion['agency_number'] = pd.to_numeric(df_stallion['agency_number'], errors='coerce')

In [ ]:
df_stallion["sku_number"] = df_stallion["sku"].str.extract('(\d+)')

# If you want to convert the extracted numbers to integer
df_stallion['sku_number'] = pd.to_numeric(df_stallion['sku_number'], errors='coerce')

In [ ]:
df_stallion_reduced = df_stallion[['volume', 'date', 'industry_volume', 'soda_volume',
       'avg_max_temp', 'price_regular', 'price_actual', 'discount',
       'avg_population_2017', 'avg_yearly_household_income_2017', 
       'discount_in_percent', 'timeseries', 'time_idx', 'month', 'log_volume',
       'avg_volume_by_sku', 'avg_volume_by_agency', 'agency_number', "sku_number"]]
df_stallion_reduced.loc[:,"date"] = pd.to_datetime(df_stallion_reduced["date"])
# Ensure nanosecond precision by casting to 'datetime64[ns]'
df_stallion_reduced.loc[:,"date"]  = df_stallion_reduced['date'].astype('datetime64[ns]')

In [ ]:
display(HTML(df_stallion_reduced.head().to_html()))

In [ ]:
max_prediction_length = 6
max_encoder_length = 24
import warnings
from darts.dataprocessing.transformers import Scaler
warnings.filterwarnings("ignore", message="Converting non-nanosecond precision datetime values to nanosecond precision")

from darts import TimeSeries
target_series = TimeSeries.from_group_dataframe(df_stallion_reduced,time_col="date", value_cols='volume', group_cols=['agency_number', "sku_number"])

# Similarly, create a TimeSeries for covariates, if needed
covariates_series = TimeSeries.from_group_dataframe(
    df_stallion_reduced, 
    time_col='date', 
    value_cols=['industry_volume', 'soda_volume', 'avg_max_temp', 'price_regular', 
                'price_actual', 'discount', 'avg_population_2017', 
                'avg_yearly_household_income_2017', 'discount_in_percent', 
                'month', 'log_volume'],
    group_cols=['agency_number', 'sku_number']
)

scaler = Scaler()
target_series_scaled = scaler.fit_transform(target_series)
covariates_series_scaled = scaler.fit_transform(covariates_series)


In [ ]:
from darts.utils.model_selection import train_test_split
target_series_scaled_train, target_series_scaled_test = train_test_split(target_series_scaled, 0.10)
covariates_series_scaled_train, covariates_series_scaled_test = train_test_split(covariates_series_scaled, 0.10)

In [ ]:
from darts.models import TransformerModel
from darts.utils.model_selection import train_test_split



# Define the Transformer model with past covariates
transformer_model = TransformerModel(
    input_chunk_length=30,       # How many past time steps the model looks at
    output_chunk_length=1,       # How many future steps to predict (e.g., next month)
    d_model=64,                  # Dimension of the model
    nhead=4,                     # Number of attention heads
    num_encoder_layers=2,        # Number of encoder layers
    num_decoder_layers=2,        # Number of decoder layers
    dropout=0.1,                 # Dropout to prevent overfitting
    batch_size=32,               # Batch size
    n_epochs=10,                # Number of epochs for training
    random_state=42              # For reproducibility
)


# Fit the model with past covariates
transformer_model.fit(
    series=target_series_scaled_train, 
    past_covariates=covariates_series_scaled_train,  # Pass the past covariates here
    verbose=True
)

In [ ]:

backtest = transformer_model.historical_forecasts(
    series=target_series_scaled_test[0],
    past_covariates=covariates_series_scaled_test[0],
    start=covariates_series_scaled_test[0].start_time(),
    forecast_horizon=1,
    retrain=False,
    verbose=True,
)
target_series_scaled_test[0].plot(label="actual")
backtest.plot(label="backtest (H=1)")



In [ ]:
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler

# Optionally scale the data
scaler = Scaler()
target_series_scaled = scaler.fit_transform(target_series)

# Define future covariates if you have them, e.g., cyclic encoders for time features like month
add_encoders = {
    'cyclic': {'future': ['month']}  # Adding future covariates automatically
}

# Define the LSTM model
lstm_model = RNNModel(
    model='LSTM',                  # Specify LSTM as the model type
    input_chunk_length=12,         # How much history to look at
    n_rnn_layers=1,                # Number of RNN layers
    dropout=0.1,                   # Dropout rate
    batch_size=32,                 # Batch size for training
    n_epochs=20,                  # Number of training epochs
    training_length=50,            # How much history is used during training
    add_encoders=add_encoders      # Automatically generate future covariates
)

# Fit the LSTM model (without past covariates, but with future encoders)
lstm_model.fit(series=target_series_scaled, verbose=True)

In [ ]:
backtest = lstm_model.historical_forecasts(
    series=target_series_scaled_test[0],
    past_covariates=covariates_series_scaled_test[0],
    start=covariates_series_scaled_test[0].start_time(),
    forecast_horizon=1,
    retrain=False,
    verbose=True,
)
target_series_scaled_test[0].plot(label="actual")
backtest.plot(label="backtest (H=1)")


In [ ]:
from pytrends.request import TrendReq
import pandas as pd

pytrend = TrendReq()

# Define the keyword you want to search for
keywords = ["CRYPTO","AI"]

# Fetch interest over time
pytrends.build_payload(keywords, cat=0, timeframe=f"{start_period} {end_period}", geo='US', gprop='')
df = pytrends.interest_over_time()

df.plot

In [ ]:
df.plot_bokeh()

In [ ]:
def collect_trend_score(keyword):
    pytrend = TrendReq() 
    pytrend.build_payload(kw_list=[keyword], timeframe='2020-01-01 2022-01-01')
    df = pytrend.interest_over_time()
    return df

In [ ]:
def resample_trend_score_df(df, keyword):
    trends = df[keyword].resample('D', convention = 'start').pad()
    trends = pd.DataFrame(trends)
    trends.rename(columns = {keyword:'trend_score'}, inplace = True)
    return trends